<h1>Vorhersage, ob es bei einem Unfall Schwerverletzte/Tote gibt</h1>

In [34]:
import geopandas as gpd
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
crash_data = gpd.read_file('../01_Source/consolidatedDataSet.geojson') 

## 1. Vorbereitung des Datensatzes

### 1.1 Vorbereitung der Datenbasis für die konkrete Verarbeitung

In [ ]:
crash_data = crash_data.drop(["ULAND","UREGBEZ","UJAHR","LINREFX","LINREFY","METER","EW2010","GFZ","HERKUNFT","NAME","KNAME","PLR_NAME",
                              "VBUS","VBUS_PERS","GESLAERM_PERS","GEMEINDE","BEZIRK","ANWERT","VERFAHRENSART","STICHTAG","BEITRAGSZUSTAND","LUMNUM","ELEM_NR","VRICHT_TXT","ZEIT_T","TAG_T"
                             ,"DURCH_T","DANN_T","DAT_T","LAT","LON","formatted_address","address_components","street_number","route","postal_code","sublocality","BRW","UART","UWOCHENTAG","ULICHTVERH","UTYP1","STRZUSTAND"], axis=1)
crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)

In [ ]:
crash_data.head()

In [35]:


#crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)


crash_data_copy = crash_data.copy()
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UART_Value"], prefix='UART_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UWOCHENTAG_Value"], prefix='UWOCHENTAG_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UTYP1_Value"], prefix='UTYP1_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["ULICHTVERH_Value"], prefix='ULICHTVERH_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["STRZUSTAND_Value"], prefix='STRZUSTAND_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["NUTZUNG"], prefix='NUTZUNG_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["GESLAERM"], prefix='GESLAERM_')

#corr_matrix = crash_data_copy.corr()
#corr_matrix["TARGET"]
crash_data_copy = crash_data_copy.drop(["geometry"], axis=1)

### 1.2 Train-Test Split

In [36]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X = crash_data_copy.drop(["TARGET"] , axis=1)
Y = crash_data_copy["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(X_train, y_train)

### 2.1 Trainieren und Testen eines RandomForestClassifiers

In [37]:
crash_data_copy["TARGET"].value_counts()

0    8586
1    1743
Name: TARGET, dtype: int64

In [38]:
(11433-2216)/11433

0.806175107145981

In [39]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier()

param_grid = {
                 'n_estimators': [5, 15, 30, 90, 100, 200],
                 'max_depth': [1, 2,3, 5, 7, 9, 11, 13],
                 'class_weight':["balanced"],
                 'min_samples_leaf': [2, 3, 4, 5, 7, 9]
             }

param_grid2 = {
                 'n_estimators': [85, 90, 95],
                 'max_depth': [3, 4, 5],
                 'min_samples_leaf': [7] #5
             }

from sklearn.model_selection import GridSearchCV

grid_clf = GridSearchCV(clf, param_grid, cv=10, verbose=10, n_jobs=5, scoring='balanced_accuracy')
grid_clf.fit(X_train, y_train)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(X_train,y_train)

y_pred=grid_clf.predict(X_test)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    2.4s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    2.5s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    2.8s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    3.1s
[Parallel(n_jobs=5)]: Done  51 tasks      | elapsed:    4.0s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:    4.5s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1876s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done  75 tasks      | elapsed:    4.7s
[Parallel(n_jobs=5)]: Done  94 tasks      | elapsed:    5.4s
[Parallel(n_jobs=5)]: Done 124 tasks      | elapsed:    6.9s
[Parallel(n_jobs=5)]: Done 154 tasks      | elapsed:    7.7s
[Parallel(n_jobs=5)]: Done 188 tasks      | elapsed:    9.6s
[Parallel(n_jobs=5)]: Done 222 tasks      | elapse

In [40]:
y_pred=grid_clf.predict(X_test)

In [41]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.670377541142304


In [42]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.27


In [43]:
grid_clf.best_params_

{'class_weight': 'balanced',
 'max_depth': 5,
 'min_samples_leaf': 9,
 'n_estimators': 30}

In [44]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1132, 573, 108, 253)

In [55]:
clf=RandomForestClassifier(random_state=42,class_weight='balanced',max_depth=7, min_samples_leaf=2, n_estimators=200)

clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [56]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.702323330106486


In [57]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1222, 483, 132, 229)

In [58]:
#Precision
tp/(tp+fp)

0.32162921348314605

In [59]:
#Recall
tp/(tp+fn)

0.6343490304709142

In [60]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.27


### 2.2. Wichtige Features für Klassifikationsentscheidung

In [61]:
feature_imp = pd.Series(clf.feature_importances_, index= X_train.columns).sort_values(ascending=False)

In [62]:
feature_imp

FATAL_CRASH_BY_DTV                                                            0.272540
UART_Value__Zusammenstoß mit vorausfahrendem / wartendem Fahrzeug             0.088219
UTYP1_Value__Überschreitenunfall                                              0.048510
IstKrad                                                                       0.040284
CRASH_BY_DTV                                                                  0.038845
UTYP1_Value__Unfall im Längsverkehr                                           0.037111
ANTEIL_KRAD                                                                   0.029923
UART_Value__Zusammenstoß zwischen Fahrzeug und Fußgänger                      0.028445
IstFuss                                                                       0.027163
ANTEIL_REISEBUSSE                                                             0.026292
UGEMEINDE_AVG_TARGET                                                          0.025965
ANTEIL_LKW                                 

In [68]:
corr_matrix = crash_data_copy.corr()
corr_matrix[corr_matrix["FATAL_CRASH_BY_DTV"] > 0.1]["FATAL_CRASH_BY_DTV"]

ANTEIL_REISEBUSSE     0.520835
CRASH_BY_DTV          0.980538
FATAL_CRASH_BY_DTV    1.000000
Name: FATAL_CRASH_BY_DTV, dtype: float64